<a href="https://colab.research.google.com/github/TheAlphaJas/A2OJ-Ladders/blob/main/Colab/StyleTTS2_Finetune_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install packages and download models

In [1]:
%%shell
git clone https://github.com/yl4579/StyleTTS2.git
cd StyleTTS2
pip install SoundFile torchaudio munch torch pydub pyyaml librosa nltk matplotlib accelerate transformers phonemizer einops einops-exts tqdm typing-extensions git+https://github.com/resemble-ai/monotonic_align.git
sudo apt-get install espeak-ng
git-lfs clone https://huggingface.co/yl4579/StyleTTS2-LibriTTS
mv StyleTTS2-LibriTTS/Models .

Cloning into 'StyleTTS2'...
remote: Enumerating objects: 372, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 372 (delta 102), reused 96 (delta 96), pack-reused 228 (from 1)
Receiving objects: 100% (372/372), 133.97 MiB | 13.82 MiB/s, done.
Resolving deltas: 100% (203/203), done.
Updating files: 100% (48/48), done.
  Cloning https://github.com/resemble-ai/monotonic_align.git to /tmp/pip-req-build-by3ns2h4
  Running command git clone --filter=blob:none --quiet https://github.com/resemble-ai/monotonic_align.git /tmp/pip-req-build-by3ns2h4
  Resolved https://github.com/resemble-ai/monotonic_align.git to commit 78b985be210a03d08bc3acc01c4df0442105366f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3

### Download dataset (LJSpeech, 200 samples, ~15 minutes of data)

You can definitely do it with fewer samples. This is just a proof of concept with 200 smaples.

In [6]:
import os
import urllib.request
import tarfile
import librosa
import soundfile as sf
import pandas as pd
from tqdm import tqdm
import yaml

# Create data directory if it doesn't exist
os.makedirs("data", exist_ok=True)

# Download LJSpeech dataset
ljspeech_url = "https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2"
ljspeech_path = "data/LJSpeech-1.1.tar.bz2"

if not os.path.exists(ljspeech_path):
    print("Downloading LJSpeech dataset...")
    urllib.request.urlretrieve(ljspeech_url, ljspeech_path)

# Extract the dataset
if not os.path.exists("data/LJSpeech-1.1"):
    print("Extracting LJSpeech dataset...")
    with tarfile.open(ljspeech_path, "r:bz2") as tar:
        tar.extractall("data")

# Upsample audio to 24kHz as required by StyleTTS2
ljspeech_dir = "data/LJSpeech-1.1"
wavs_dir = os.path.join(ljspeech_dir, "wavs")
upsampled_dir = os.path.join(ljspeech_dir, "wavs_24kHz")
os.makedirs(upsampled_dir, exist_ok=True)

# Create metadata file paths
metadata_path = os.path.join(ljspeech_dir, "metadata.csv")
train_list_path = "Data/train_list.txt"
val_list_path = "Data/val_list.txt"

# Create Data directory
os.makedirs("Data", exist_ok=True)

# Read metadata
metadata = pd.read_csv(metadata_path, sep="|", header=None,
                      names=["file_id", "transcription", "normalized_transcription"])

# Upsample audio files to 24kHz
print("Upsampling audio files to 24kHz...")
for idx, row in tqdm(metadata.iterrows(), total=len(metadata)):
    file_id = row["file_id"]
    wav_path = os.path.join(wavs_dir, f"{file_id}.wav")
    upsampled_path = os.path.join(upsampled_dir, f"{file_id}.wav")

    if not os.path.exists(upsampled_path):
        # Load audio and resample to 24kHz
        audio, sr = librosa.load(wav_path, sr=None)
        audio_24k = librosa.resample(audio, orig_sr=sr, target_sr=24000)
        sf.write(upsampled_path, audio_24k, 24000)

# Split into training and validation sets (95% train, 5% validation)
split_idx = int(len(metadata) * 0.95)
train_metadata = metadata.iloc[:split_idx]
val_metadata = metadata.iloc[split_idx:]

# Create train list file
with open(train_list_path, "w") as f:
    for idx, row in train_metadata.iterrows():
        file_id = row["file_id"]
        normalized_text = row["normalized_transcription"]
        f.write(f"{file_id}.wav|{normalized_text}\n")

# Create validation list file
with open(val_list_path, "w") as f:
    for idx, row in val_metadata.iterrows():
        file_id = row["file_id"]
        normalized_text = row["normalized_transcription"]
        f.write(f"{file_id}.wav|{normalized_text}\n")

print(f"Dataset prepared successfully!")
print(f"Training samples: {len(train_metadata)}")
print(f"Validation samples: {len(val_metadata)}")

Extracting LJSpeech dataset...
Upsampling audio files to 24kHz...


100%|██████████| 13100/13100 [02:42<00:00, 80.73it/s]


Dataset prepared successfully!
Training samples: 12445
Validation samples: 655


In [4]:
%cd StyleTTS2
# !rm -rf Data

/content/StyleTTS2


In [ ]:
# !gdown --id 1vqz26D3yn7OXS2vbfYxfSnpLS6m6tOFP
# !unzip Data.zip

### Change the finetuning config

Depending on the GPU you got, you may want to change the bacth size, max audio length, epiochs and so on.

In [ ]:
# data_params:
#   train_data: "Data/train_list.txt"
#   val_data: "Data/val_list.txt"
#   root_path: "/local/LJSpeech-1.1/wavs"
#   OOD_data: "Data/OOD_texts.txt"
#   min_length: 50 # sample until texts with this size are obtained for OOD texts

In [5]:
config_path = "Configs/config_ft.yml"

import yaml
config = yaml.safe_load(open(config_path))

In [7]:
config['data_params']['root_path'] = "data/LJSpeech-1.1/wavs_24kHz"

config['batch_size'] = 2 # not enough RAM
config['max_len'] = 100 # not enough RAM
config['loss_params']['joint_epoch'] = 110 # we do not do SLM adversarial training due to not enough RAM

with open(config_path, 'w') as outfile:
  yaml.dump(config, outfile, default_flow_style=True)

### Start finetuning


In [15]:
!python train_finetune.py --config_path ./Configs/config_ft.yml

2025-04-09 23:15:43.292514: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744240543.313059   11438 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744240543.319307   11438 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-09 23:15:43.340590: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
yes
config.json: 100% 2.23k/2.23k [00:00<00:00, 15.6MB/s]
pytorch_model.bin: 100% 378M/378M [00:02<00:00, 153MB/s] 
m

### Test the model quality

Note that this mainly serves as a proof of concept due to RAM limitation of free Colab instances. A lot of settings are suboptimal. In the future when DDP works for train_second.py, we will also add mixed precision finetuning to save time and RAM. You can also add SLM adversarial training run if you have paid Colab services (such as A100 with 40G of RAM).

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

import random
random.seed(0)

import numpy as np
np.random.seed(0)

# load packages
import time
import random
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
import librosa
from nltk.tokenize import word_tokenize

from models import *
from utils import *
from text_utils import TextCleaner
textclenaer = TextCleaner()

%matplotlib inline

to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300)
mean, std = -4, 4

def length_to_mask(lengths):
    mask = torch.arange(lengths.max()).unsqueeze(0).expand(lengths.shape[0], -1).type_as(lengths)
    mask = torch.gt(mask+1, lengths.unsqueeze(1))
    return mask

def preprocess(wave):
    wave_tensor = torch.from_numpy(wave).float()
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor

def compute_style(path):
    wave, sr = librosa.load(path, sr=24000)
    audio, index = librosa.effects.trim(wave, top_db=30)
    if sr != 24000:
        audio = librosa.resample(audio, sr, 24000)
    mel_tensor = preprocess(audio).to(device)

    with torch.no_grad():
        ref_s = model.style_encoder(mel_tensor.unsqueeze(1))
        ref_p = model.predictor_encoder(mel_tensor.unsqueeze(1))

    return torch.cat([ref_s, ref_p], dim=1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load phonemizer
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='en-us', preserve_punctuation=True,  with_stress=True)

config = yaml.safe_load(open("Models/LJSpeech/config_ft.yml"))

# load pretrained ASR model
ASR_config = config.get('ASR_config', False)
ASR_path = config.get('ASR_path', False)
text_aligner = load_ASR_models(ASR_path, ASR_config)

# load pretrained F0 model
F0_path = config.get('F0_path', False)
pitch_extractor = load_F0_models(F0_path)

# load BERT model
from Utils.PLBERT.util import load_plbert
BERT_path = config.get('PLBERT_dir', False)
plbert = load_plbert(BERT_path)

model_params = recursive_munch(config['model_params'])
model = build_model(model_params, text_aligner, pitch_extractor, plbert)
_ = [model[key].eval() for key in model]
_ = [model[key].to(device) for key in model]

In [ ]:
files = [f for f in os.listdir("Models/LJSpeech/") if f.endswith('.pth')]
sorted_files = sorted(files, key=lambda x: int(x.split('_')[-1].split('.')[0]))

In [ ]:
params_whole = torch.load("Models/LJSpeech/" + sorted_files[-1], map_location='cpu')
params = params_whole['net']

In [ ]:
for key in model:
    if key in params:
        print('%s loaded' % key)
        try:
            model[key].load_state_dict(params[key])
        except:
            from collections import OrderedDict
            state_dict = params[key]
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():
                name = k[7:] # remove `module.`
                new_state_dict[name] = v
            # load params
            model[key].load_state_dict(new_state_dict, strict=False)
#             except:
#                 _load(params[key], model[key])
_ = [model[key].eval() for key in model]

In [ ]:
from Modules.diffusion.sampler import DiffusionSampler, ADPM2Sampler, KarrasSchedule

In [ ]:
sampler = DiffusionSampler(
    model.diffusion.diffusion,
    sampler=ADPM2Sampler(),
    sigma_schedule=KarrasSchedule(sigma_min=0.0001, sigma_max=3.0, rho=9.0), # empirical parameters
    clamp=False
)

In [ ]:
def inference(text, ref_s, alpha = 0.3, beta = 0.7, diffusion_steps=5, embedding_scale=1):
    text = text.strip()
    ps = global_phonemizer.phonemize([text])
    ps = word_tokenize(ps[0])
    ps = ' '.join(ps)
    tokens = textclenaer(ps)
    tokens.insert(0, 0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(device)
        text_mask = length_to_mask(input_lengths).to(device)

        t_en = model.text_encoder(tokens, input_lengths, text_mask)
        bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
        d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

        s_pred = sampler(noise = torch.randn((1, 256)).unsqueeze(1).to(device),
                                          embedding=bert_dur,
                                          embedding_scale=embedding_scale,
                                            features=ref_s, # reference from the same speaker as the embedding
                                             num_steps=diffusion_steps).squeeze(1)


        s = s_pred[:, 128:]
        ref = s_pred[:, :128]

        ref = alpha * ref + (1 - alpha)  * ref_s[:, :128]
        s = beta * s + (1 - beta)  * ref_s[:, 128:]

        d = model.predictor.text_encoder(d_en,
                                         s, input_lengths, text_mask)

        x, _ = model.predictor.lstm(d)
        duration = model.predictor.duration_proj(x)

        duration = torch.sigmoid(duration).sum(axis=-1)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1)

        pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
        c_frame = 0
        for i in range(pred_aln_trg.size(0)):
            pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].data)] = 1
            c_frame += int(pred_dur[i].data)

        # encode prosody
        en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(en)
            asr_new[:, :, 0] = en[:, :, 0]
            asr_new[:, :, 1:] = en[:, :, 0:-1]
            en = asr_new

        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)

        asr = (t_en @ pred_aln_trg.unsqueeze(0).to(device))
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(asr)
            asr_new[:, :, 0] = asr[:, :, 0]
            asr_new[:, :, 1:] = asr[:, :, 0:-1]
            asr = asr_new

        out = model.decoder(asr,
                                F0_pred, N_pred, ref.squeeze().unsqueeze(0))


    return out.squeeze().cpu().numpy()[..., :-50] # weird pulse at the end of the model, need to be fixed later

### Synthesize speech

In [ ]:
text = '''Maltby and Company would issue warrants on them deliverable to the importer, and the goods were then passed to be stored in neighboring warehouses.
'''

In [ ]:
# get a random reference in the training set, note that it doesn't matter which one you use
path = "Data/wavs/LJ001-0110.wav"
# this style vector ref_s can be saved as a parameter together with the model weights
ref_s = compute_style(path)

In [ ]:
start = time.time()
wav = inference(text, ref_s, alpha=0.9, beta=0.9, diffusion_steps=10, embedding_scale=1)
rtf = (time.time() - start) / (len(wav) / 24000)
print(f"RTF = {rtf:5f}")
import IPython.display as ipd
display(ipd.Audio(wav, rate=24000, normalize=False))